    Problem Statement.

    Table: Transactions

    +----------------+---------+
    | Column Name    | Type    |
    +----------------+---------+
    | id             | int     |
    | country        | varchar |
    | state          | enum    |
    | amount         | int     |
    | trans_date     | date    |
    +----------------+---------+
    id is the primary key of this table.
    The table has information about incoming transactions.
    The state column is an enum of type ["approved", "declined"].

    Table: Chargebacks

    +----------------+---------+
    | Column Name    | Type    |
    +----------------+---------+
    | trans_id       | int     |
    | trans_date     | date    |
    +----------------+---------+
    Chargebacks contains basic information regarding incoming chargebacks from some transactions placed in Transactions table.
    trans_id is a foreign key to the id column of Transactions table.
    Each chargeback corresponds to a transaction made previously even if they were not approved.



    Write an SQL query to find for each month and country: the number of approved transactions and their total amount, the number of chargebacks, and their total amount.

    Note: In your query, given the month and country, ignore rows with all zeros.

    Return the result table in any order.

    The query result format is in the following example.



    Example 1:

    Input: 
    Transactions table:
    +-----+---------+----------+--------+------------+
    | id  | country | state    | amount | trans_date |
    +-----+---------+----------+--------+------------+
    | 101 | US      | approved | 1000   | 2019-05-18 |
    | 102 | US      | declined | 2000   | 2019-05-19 |
    | 103 | US      | approved | 3000   | 2019-06-10 |
    | 104 | US      | declined | 4000   | 2019-06-13 |
    | 105 | US      | approved | 5000   | 2019-06-15 |
    +-----+---------+----------+--------+------------+
    Chargebacks table:
    +----------+------------+
    | trans_id | trans_date |
    +----------+------------+
    | 102      | 2019-05-29 |
    | 101      | 2019-06-30 |
    | 105      | 2019-09-18 |
    +----------+------------+
    Output: 
    +---------+---------+----------------+-----------------+------------------+-------------------+
    | month   | country | approved_count | approved_amount | chargeback_count | chargeback_amount |
    +---------+---------+----------------+-----------------+------------------+-------------------+
    | 2019-05 | US      | 1              | 1000            | 1                | 2000              |
    | 2019-06 | US      | 2              | 8000            | 1                | 1000              |
    | 2019-09 | US      | 0              | 0               | 1                | 5000              |
    +---------+---------+----------------+-----------------+--------------------------------------

# WIth

In [ ]:
# Write your MySQL query statement below
with transaction_counts as 
(select left(trans_date, 7) as month, country, count(*) as approved_count,
sum(amount) as approved_amount
from Transactions
where state='approved'
group by month, country),

chargeback_counts as 
(select left(c.trans_date, 7) as month, country, count(*) as chargeback_count,
sum(amount) as chargeback_amount 
from Chargebacks c
join Transactions t on c.trans_id = t.id
group by month, country)


select month, country, sum(approved_count) as approved_count, 
sum(approved_amount)as approved_amount, sum(chargeback_count)as chargeback_count, 
sum(chargeback_amount)as chargeback_amount
from
(select t.*, coalesce(chargeback_count, 0) as chargeback_count, coalesce(chargeback_amount, 0) as chargeback_amount
from transaction_counts t 
left join chargeback_counts c on t.month = c.month and t.country = c.country

union 

select c.month, c.country, coalesce(approved_count, 0) as approved_count, coalesce(approved_amount, 0) as approved_amount, c.chargeback_count, c.chargeback_amount
from transaction_counts t 
right join chargeback_counts c on t.month = c.month and t.country = c.country) tab
group by month, country

# Union

In [ ]:
SELECT month, country, 
SUM(CASE WHEN state = "approved" THEN 1 ELSE 0 END) AS approved_count, 
SUM(CASE WHEN state = "approved" THEN amount ELSE 0 END) AS approved_amount, 
SUM(CASE WHEN state = "back" THEN 1 ELSE 0 END) AS chargeback_count, 
SUM(CASE WHEN state = "back" THEN amount ELSE 0 END) AS chargeback_amount
FROM
(
    SELECT LEFT(chargebacks.trans_date, 7) AS month, country, "back" AS state, amount
    FROM chargebacks
    JOIN transactions ON chargebacks.trans_id = transactions.id
    UNION ALL
    SELECT LEFT(trans_date, 7) AS month, country, state, amount
    FROM transactions
    WHERE state = "approved"
) s
GROUP BY month, country